In [1]:
# 1. 必要なライブラリをインストール
!pip install pandas requests tqdm lxml


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# 2. メインのPythonコード
import pandas as pd
import requests
import time
from tqdm import tqdm
import io

def create_debug_file():
    """
    【デバッグ専用】スクレイピングした生のデータをCSVファイルに出力する。
    """
    print("Yahoo!ファイナンスの時価総額ランキングからデバッグ用の生データを取得します...")
    start_time = time.time()
    
    base_url = "https://finance.yahoo.co.jp/stocks/ranking/marketCapitalHigh?market=all&page={}"
    all_dfs = []
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    }
    
    max_page = 5 # デバッグなので、ページ数を5に減らして高速化します
    for page in tqdm(range(1, max_page + 1), desc="ランキングページ取得中"):
        try:
            url = base_url.format(page)
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            tables = pd.read_html(io.StringIO(response.text))
            
            if len(tables) > 0:
                all_dfs.append(tables[0])
            else:
                break
            time.sleep(0.5)
        except Exception as e:
            print(f"ページ {page} の取得中にエラーが発生しました: {e}")
            break

    if not all_dfs:
        print("データを取得できませんでした。")
        return

    # --- 取得した全データを結合 ---
    full_df = pd.concat(all_dfs, ignore_index=True)

    # ★★★ ここが今回の目的 ★★★
    # 取得した生のDataFrameを、一切加工せずにCSVファイルとして保存する
    csv_filename = 'raw_scraping_data.csv'
    try:
        # encoding='utf-8-sig' はExcelで文字化けしないためのおまじない
        full_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        print(f"\n--- 【デバッグ用ファイル出力完了】 ---")
        print(f"スクレイピングした生データを '{csv_filename}' に保存しました。")
        print("\n★★★ 次のステップ ★★★")
        print("1. Google Colabの左側にあるフォルダアイコンをクリックしてファイル一覧を表示してください。")
        print(f"2. '{csv_filename}' というファイルが表示されているはずです。")
        print("3. このファイルをダウンロードし、Excelやテキストエディタで開いてください。")
        print("4. 特に `'名称・コード・市場'` 列のセルの中身が、**具体的にどのような文字列になっているか**を確認し、その内容を教えてください。")
        print("（例：「トヨタ自動車(株) 7203 東証PRM 掲示板」のようにスペース区切りか、「トヨタ自動車(株)7203東証PRM掲示板」のように連結されているか、など）")

    except Exception as e:
        print(f"\nCSVファイルへの保存中にエラーが発生しました: {e}")

    end_time = time.time()
    print(f"\n処理が完了しました。所要時間: {end_time - start_time:.2f} 秒")


if __name__ == '__main__':
    create_debug_file()

Yahoo!ファイナンスの時価総額ランキングからデバッグ用の生データを取得します...


ランキングページ取得中: 100%|██████████| 5/5 [00:04<00:00,  1.15it/s]


--- 【デバッグ用ファイル出力完了】 ---
スクレイピングした生データを 'raw_scraping_data.csv' に保存しました。

★★★ 次のステップ ★★★
1. Google Colabの左側にあるフォルダアイコンをクリックしてファイル一覧を表示してください。
2. 'raw_scraping_data.csv' というファイルが表示されているはずです。
3. このファイルをダウンロードし、Excelやテキストエディタで開いてください。
4. 特に `'名称・コード・市場'` 列のセルの中身が、**具体的にどのような文字列になっているか**を確認し、その内容を教えてください。
（例：「トヨタ自動車(株) 7203 東証PRM 掲示板」のようにスペース区切りか、「トヨタ自動車(株)7203東証PRM掲示板」のように連結されているか、など）

処理が完了しました。所要時間: 4.37 秒
